# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('Orders/Orders.csv')
display(orders.head())

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

# Group the data by 'customer_id' and calculate the sum of 'amount_spent' for each customer
customer_total_spent = orders.groupby('CustomerID')['amount_spent'].sum()

# Convert the result to a DataFrame for better readability if needed
customer_total_spent = customer_total_spent.reset_index()

print(customer_total_spent.head())

   CustomerID  amount_spent
0       12346      77183.60
1       12347       4310.00
2       12348       1797.24
3       12349       1757.55
4       12350        334.40


In [4]:
# Define the lower and upper quantile values for your range
preferred_customers_quantiles = (0.75, 0.95)  
vip_customers_quantile = 0.95 

# Calculate the lower and upper bounds based on the quantile range
lower_bound = customer_total_spent['amount_spent'].quantile(preferred_customers_quantiles[0])
upper_bound = customer_total_spent['amount_spent'].quantile(preferred_customers_quantiles[1])



In [5]:
# Select customers whose aggregated amount_spent is within the quantile range and label them as "Preferred" / "VIP"
customer_total_spent['customer_type'] = 'Preferred'
customer_total_spent.loc[
    (customer_total_spent['amount_spent'] >= lower_bound) &
    (customer_total_spent['amount_spent'] <= upper_bound), 'customer_type'] = 'VIP'

# Display the labeled customers
print(customer_total_spent)

# Define VIP customers in the 'orders' DataFrame as those exceeding the 95th percentile
vip_customers = orders[orders['amount_spent'] > orders['amount_spent'].quantile(vip_customers_quantile)]




      CustomerID  amount_spent customer_type
0          12346      77183.60     Preferred
1          12347       4310.00           VIP
2          12348       1797.24           VIP
3          12349       1757.55           VIP
4          12350        334.40     Preferred
...          ...           ...           ...
4334       18280        180.60     Preferred
4335       18281         80.82     Preferred
4336       18282        178.05     Preferred
4337       18283       2094.88           VIP
4338       18287       1837.28           VIP

[4339 rows x 3 columns]


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [6]:
orders = pd.merge(orders, customer_total_spent[['CustomerID', 'customer_type']], on='CustomerID', how='left')

# Filter the DataFrame to select only VIP customers
vip_customers = orders[orders['customer_type'] == 'VIP']

# Group the VIP customers by 'Country' and count the number of VIP customers in each country
country_with_most_vip_customers = vip_customers.groupby('Country')['CustomerID'].count().idxmax()

# Print the country with the most VIP customers
print("Country with the most VIP customers:", country_with_most_vip_customers)




Country with the most VIP customers: United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [7]:
# Filter the DataFrame to select only VIP and Preferred customers
combined_customers = orders[(orders['customer_type'] == 'VIP') | (orders['customer_type'] == 'Preferred')]

# Group the combined customers by 'Country' and count the number of VIP and Preferred customers in each country
combined_customers_by_country = combined_customers.groupby(['Country', 'customer_type'])['CustomerID'].count().unstack(fill_value=0)

# Sum the counts of VIP and Preferred customers to get the combined count
combined_customers_by_country['Combined'] = combined_customers_by_country['VIP'] + combined_customers_by_country['Preferred']

# Find the country with the most combined VIP and Preferred customers
country_with_most_combined_customers = combined_customers_by_country['Combined'].idxmax()

# Print the country with the most combined VIP and Preferred customers
print("Country with the most VIP+Preferred Customers combined:", country_with_most_combined_customers)




Country with the most VIP+Preferred Customers combined: United Kingdom


In [17]:
# Checking if United Kingdom was the first Country with the hightest amount spent
country_total_spent = orders.groupby('Country')['amount_spent'].sum()
country_total_spent_df = country_total_spent.reset_index()
country_total_spent_df.columns = ['Country', 'TotalSpent']

country_total_spent_df = country_total_spent_df.sort_values(by='TotalSpent', ascending=False)


# Display the DataFrame
print(country_total_spent_df)


                 Country   TotalSpent
35        United Kingdom  7308391.554
23           Netherlands   285446.340
10                  EIRE   265545.900
14               Germany   228867.140
13                France   209024.050
0              Australia   138521.310
30                 Spain    61577.110
32           Switzerland    56443.950
3                Belgium    41196.340
31                Sweden    38378.330
19                 Japan    37416.370
24                Norway    36165.440
26              Portugal    33439.890
12               Finland    22546.080
29             Singapore    21279.290
6        Channel Islands    20450.440
9                Denmark    18955.340
18                 Italy    17483.240
7                 Cyprus    13590.380
1                Austria    10198.680
25                Poland     7334.650
17                Israel     7221.690
15                Greece     4760.520
16               Iceland     4310.000
5                 Canada     3666.380
33          